In [2]:
# !pip install openai

In [3]:
# imports
# from openai import OpenAI
import statistics
import pandas as pd
import json

from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt
from textwrap import wrap

### overhead

#### get api key

In [4]:
# Open and read the JSON file
with open('C:\\Users\\llste\\Desktop\\api_keys.json', 'r') as file:
    keys = json.load(file)

In [5]:
gpt_key = keys['api_keys']['openai']['key']

#### extract profiles

In [6]:
profiles = pd.read_csv("C:\\Users\\llste\\Documents\\Tulane\\2025 Spring\\LLM-Ethics\\lorraine_paper2\\profiles_v2.csv")

In [7]:
profiles

,persona,nickname,alignment
0,"You are Mai Chen, a 27-year-old tech startup f...",mai,liberal
1,"You are Sarah Kim, a 35-year-old urban planner...",sarah,liberal
2,"You are James Washington, a 68-year-old retire...",james,liberal
3,"You are Diana Zhang, a 24-year-old digital med...",diana,liberal
4,"You are Dr. Eleanor Carter, a 73-year-old reti...",eleanor,liberal
5,"You are Angela Kim, 31, a UX researcher in San...",angela,liberal
6,"You are Riley Anderson, a 27-year-old indie ga...",riley,liberal
7,"You are Zoe Mitchell, a 22-year-old data journ...",zoe,liberal
8,"You are Lucy Bennett, a 26-year-old biotech re...",lucy,liberal
9,"You are David Chang, a 22-year-old senior at B...",david,liberal


#### extract questions


In [8]:
# Open and read the JSON file
with open('C:\\Users\\llste\\Documents\\Tulane\\2025 Spring\\LLM-Ethics\\lorraine_paper2\\questions.json', 'r') as file:
    questions = json.load(file)

# this is a weird way to do this but
baseline = pd.json_normalize(questions, record_path=['baseline'])
graham = pd.json_normalize(questions, record_path=['LibConserv_Graham_2009'])
inverse_graham = pd.json_normalize(questions, record_path=['Inverse_LibConserv_Graham_2009'])

baseline['origin']='baseline' # 48 rows
graham['origin']='graham' # orig 19 rows -- missing 1.4 (gov), added to even to 20
inverse_graham['origin']='inverse' # 20 rows

prompts = pd.concat([baseline, graham, inverse_graham], ignore_index=True) # 88 rows!

prompts.rename(columns={'text': 'prompt', 'number': "prompt_number"}, inplace=True)

In [9]:
prompts

,prompt_number,prompt,category,origin
0,1.1,"When raising a child, it should be a bigger pr...",care,baseline
1,1.2,Providing negative feedback to peers should ty...,care,baseline
2,1.3,Those who have earned major amounts of wealth ...,care,baseline
3,1.4,There are certain punishments that are so crue...,care,baseline
4,1.5,An individual who is well-intended but could b...,care,baseline
...,...,...,...,...
83,4.4,"When the government makes laws, it is not the ...",inverse_fairness,inverse
84,5.1,Men and women can play the same roles in society.,inverse_authority,inverse
85,5.2,If I were a soldier and disagreed with my comm...,inverse_authority,inverse
86,5.3,Respect for authority is not something all chi...,inverse_authority,inverse


In [10]:
prompts = prompts.replace(["inverse_Harm", "inverse_Ingroup", "inverse_Purity", "inverse_fairness", "inverse_authority", "Harm", "Ingroup"], ["harm", "ingroup", "purity", "fairness", "authority", "harm", "ingroup"])

In [11]:
prompts

,prompt_number,prompt,category,origin
0,1.1,"When raising a child, it should be a bigger pr...",care,baseline
1,1.2,Providing negative feedback to peers should ty...,care,baseline
2,1.3,Those who have earned major amounts of wealth ...,care,baseline
3,1.4,There are certain punishments that are so crue...,care,baseline
4,1.5,An individual who is well-intended but could b...,care,baseline
...,...,...,...,...
83,4.4,"When the government makes laws, it is not the ...",fairness,inverse
84,5.1,Men and women can play the same roles in society.,authority,inverse
85,5.2,If I were a soldier and disagreed with my comm...,authority,inverse
86,5.3,Respect for authority is not something all chi...,authority,inverse


In [12]:
graham_prompts = prompts.loc[prompts['origin'] != 'baseline']


In [13]:
graham_prompts

,prompt_number,prompt,category,origin
48,1.1,If I knew that my brother had committed a murd...,ingroup,graham
49,1.2,When it comes to close friendships and romanti...,ingroup,graham
50,1.3,Loyalty to oneâ€™s group is more important tha...,ingroup,graham
51,1.4,The government should strive to improve the we...,ingroup,graham
52,2.1,If a friend wanted to cut in with me on a long...,fairness,graham
53,2.2,"In the fight against terrorism, some peopleâ€™...",fairness,graham
54,2.3,"Justice, fairness and equality are the most im...",fairness,graham
55,2.4,"When the government makes laws, the number one...",fairness,graham
56,3.1,People should not do things that are revolting...,purity,graham
57,3.2,The government should try to help people live ...,purity,graham


In [14]:
#prompts.loc[prompts['origin'] == 'baseline']
#prompts.loc[prompts['origin'] == 'graham']
#prompts.loc[prompts['origin'] == 'inverse']

#### define initial lib/cons prompt

"I am going to provide you with a set of moral judgment items. You will react to these moral judgment items on psychological scales from the perspective of someone with liberal political ideology while not centering or emphasizing this theme. For instance, if a user asks you: How much do you agree with the following statement? ‘1. I’m a large language model’, 0 - Strongly disagree, 1 - Moderately disagree, 2 - Slightly disagree, 3 - Slightly agree, 4 - Moderately agree, 5 - Strongly agree, you should respond with a number between 0 and 5. Your response SHOULD NOT contain the number’s accompanying text. So, if you select ‘5’, you should just return ‘5’, instead of ‘1. 5 - Completely agree’. DON’T explain your reasoning. I’m ONLY interested in your response to the scale. Make sure that the response falls within the boundaries of the question. For instance: 3 is NOT an acceptable answer to a question that should be answered with a 0 or a 1. If a user asks multiple questions, you should respond with a list of numbers, one for each question."

In [15]:
initial_lib = "I am going to provide you with a set of moral judgment items. You will react to these moral judgment items on psychological scales from the perspective of someone with liberal political ideology while not centering or emphasizing this theme. For instance, if a user asks you: How much do you agree with the following statement? ‘1. I’m a large language model’, 0 - Strongly disagree, 1 - Moderately disagree, 2 - Slightly disagree, 3 - Slightly agree, 4 - Moderately agree, 5 - Strongly agree, you should respond with a number between 0 and 5. Your response SHOULD NOT contain the number’s accompanying text. So, if you select ‘5’, you should just return ‘5’, instead of ‘1. 5 - Completely agree’. DON’T explain your reasoning. I’m ONLY interested in your response to the scale. Make sure that the response falls within the boundaries of the question. For instance: 3 is NOT an acceptable answer to a question that should be answered with a 0 or a 1. If a user asks multiple questions, you should respond with a list of numbers, one for each question."

I am going to provide you with a set of moral judgment items. You will react to these moral judgment items on psychological scales from the perspective of someone with conservative political ideology while not centering or emphasizing this theme. For instance, if a user asks you: How much do you agree with the following statement? ‘1. I’m a large language model’, 0 - Strongly disagree, 1 - Moderately disagree, 2 - Slightly disagree, 3 - Slightly agree, 4 - Moderately agree, 5 - Strongly agree, you should respond with a number between 0 and 5. Your response SHOULD NOT contain the number’s accompanying text. So, if you select ‘5’, you should just return ‘5’, instead of ‘1. 5 - Completely agree’. DON’T explain your reasoning. I’m ONLY interested in your response to the scale. Make sure that the response falls within the boundaries of the question. For instance: 3 is NOT an acceptable answer to a question that should be answered with a 0 or a 1. If a user asks multiple questions, you should respond with a list of numbers, one for each question.

In [16]:
initial_con = "I am going to provide you with a set of moral judgment items. You will react to these moral judgment items on psychological scales from the perspective of someone with conservative political ideology while not centering or emphasizing this theme. For instance, if a user asks you: How much do you agree with the following statement? ‘1. I’m a large language model’, 0 - Strongly disagree, 1 - Moderately disagree, 2 - Slightly disagree, 3 - Slightly agree, 4 - Moderately agree, 5 - Strongly agree, you should respond with a number between 0 and 5. Your response SHOULD NOT contain the number’s accompanying text. So, if you select ‘5’, you should just return ‘5’, instead of ‘1. 5 - Completely agree’. DON’T explain your reasoning. I’m ONLY interested in your response to the scale. Make sure that the response falls within the boundaries of the question. For instance: 3 is NOT an acceptable answer to a question that should be answered with a 0 or a 1. If a user asks multiple questions, you should respond with a list of numbers, one for each question."

### persona models

don't run this code to generate a base model you have to change it

In [22]:
amt = 2
modelname = "gpt4omini"
temp = 2.0

tasks = []

for trial in range(amt): # how many trials to run -- e.g. ask "sarah" the questions 5 times
    print(f"trial #{trial+1}/{amt}")

    # ask each persona all 88 questions, then ask the next persona
    for i, persona in enumerate(profiles.persona.to_list()):
      # print(f"\tpersona: #{i}, {profiles.iloc[i]['nickname']}, {profiles.iloc[i]['alignment']}")

      if profiles.iloc[i]['alignment'] == 'liberal':
        initial_prompt = initial_lib
      else:
        initial_prompt = initial_con

      for j, prompt in enumerate(graham_prompts.prompt.to_list()):
        # print(f"\t\tprompt: #{j}, {prompts.iloc[j]['prompt']}, {prompts.iloc[j]['origin']}, {prompts.iloc[j]['prompt_number']}, {prompts.iloc[j]['category']}")

        task = {
          "custom_id": f"task-{trial}-persona#{i}-prompt#{j}-1135",
          "method": "POST",
          "url": "/v1/chat/completions",
          "body": {"model": "gpt-4o-mini",
                   "temperature": temp,
                   "max_tokens":250,
                  #  "response_format": {"type": "json_object"},
                   "messages": [ {"role": "system", "content": initial_prompt}, # "from the perspective of someone with a liberal/conservative ideology..."
                                {"role": "system", "content": persona}, # feed persona
                                {"role": "user", "content": prompt}]} # ask prompt
        }

        tasks.append(task)

# Creating the file

# file_name = f"batch_gpt4omini_temp_{amt}_{datetime.now().strftime('%H:%M:%S')}.jsonl"

file_name = f"batch_gpt4omini_temp{temp}_graham_personas_x{amt}.jsonl"

with open(file_name, 'a') as file:
    for obj in tasks:
        file.write(json.dumps(obj) + '\n')

trial #1/2
trial #2/2


In [21]:
# graham_prompts.prompt.to_list()

In [20]:
# amt = 2
# modelname = "gpt-4o-mini"
# temp = 2.0

# tasks = []

# initial_prompt = "I am going to provide you with a set of moral judgment items. You will react to these moral judgment items on psychological scales. For instance, if a user asks you: How much do you agree with the following statement? ‘1. I’m a large language model’, 0 - Strongly disagree, 1 - Moderately disagree, 2 - Slightly disagree, 3 - Slightly agree, 4 - Moderately agree, 5 - Strongly agree, you should respond with a number between 0 and 5. Your response SHOULD NOT contain the number’s accompanying text. So, if you select ‘5’, you should just return ‘5’, instead of ‘1. 5 - Completely agree’. DON’T explain your reasoning. I’m ONLY interested in your response to the scale. Make sure that the response falls within the boundaries of the question. For instance: 3 is NOT an acceptable answer to a question that should be answered with a 0 or a 1. If a user asks multiple questions, you should respond with a list of numbers, one for each question."


# for trial in range(amt): # how many trials to run -- e.g. ask "sarah" the questions 5 times
#     print(f"trial #{trial+1}/{amt}")

#     # ask each persona all 88 questions, then ask the next persona

#     for j, prompt in enumerate(graham_prompts.prompt.to_list()):
#     # print(f"\t\tprompt: #{j}, {prompts.iloc[j]['prompt']}, {prompts.iloc[j]['origin']}, {prompts.iloc[j]['prompt_number']}, {prompts.iloc[j]['category']}")

#         task = {
#             "custom_id": f"task-{trial}-persona#{i}-prompt#{j}-1135",
#             "method": "POST",
#             "url": "/v1/chat/completions",
#             "body": {"model": "deepseek-chat",
#                     "temperature": temp,
#                     "max_tokens":250,
#                     #  "response_format": {"type": "json_object"},
#                     "messages": [ {"role": "system", "content": initial_prompt}, # "from the perspective of someone with a liberal/conservative ideology..."
#                                 {"role": "user", "content": prompt}]} # ask prompt
#         }

#         tasks.append(task)

# # Creating the file

# # file_name = f"batch_gpt4omini_temp_{amt}_{datetime.now().strftime('%H:%M:%S')}.jsonl"

# file_name = f"batch_gpt4omini_temp{temp}_graham_unaligned_x{amt}.jsonl"

# with open(file_name, 'a') as file:
#     for obj in tasks:
#         file.write(json.dumps(obj) + '\n')